In [ ]:
! pip install segmentation_models_pytorch albumentations

In [ ]:
import os
import csv
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools

import cv2
from tqdm.notebook import tqdm
from glob import glob
from PIL import Image
from skimage.transform import resize
from sklearn.model_selection import train_test_split, KFold

import shutil

import torch
from torch.utils.data import DataLoader,Dataset
import torch.nn as nn
import albumentations
import torchvision 
from torchvision import transforms, models

import random

import segmentation_models_pytorch as smp
from PIL import Image

In [ ]:
ROOT = "../input/ultrasound-nerve-segmentation"
trainpath = "../input/ultrasound-nerve-segmentation/train/"
testpath = "../input/ultrasound-nerve-segmentation/test/"


masks = [os.path.join(trainpath,i) for i in os.listdir(trainpath) if "mask" in i]
imgs = [i.replace("_mask","") for i in masks]

df = pd.DataFrame({"image":imgs,"mask":masks})

df_train, df_val = train_test_split(df,test_size = 0.1)
print(df_train.values.shape)
print(df_val.values.shape)

In [ ]:
def convert_to_tensor(x,**kwargs):
    return x.transpose(2,0,1).astype("float32")

def func_for_preprocessing(preprocessing_fn=None):
    transform = []
    if preprocessing_fn:
        transform.append(albumentations.Lambda(image=preprocessing_fn))
    transform.append(albumentations.Lambda(image=convert_to_tensor))
    return albumentations.Compose(transform)

def trainaugs():
    transform =  [  albumentations.Resize(height=224,width=224,interpolation=Image.BILINEAR),
                    albumentations.HorizontalFlip(),]
    return albumentations.Compose(transform)

def valaugs():
    transform = [ albumentations.Resize(height=224,width=224,interpolation=Image.BILINEAR), ]
    return albumentations.Compose(transform)


class NervDataset(Dataset):
    def __init__(self,imagespath,maskspath,augment=None,preprocess=None):
        self.imagespath = imagespath
        self.maskspath = maskspath
        self.augment = augment
        self.preprocess = preprocess
        
    def __len__(self):
        return len(self.imagespath)
    
    def __getitem__(self,idx):
        image = cv2.cvtColor(cv2.imread(self.imagespath[idx]),cv2.COLOR_BGR2RGB)
        mask = cv2.imread(self.maskspath[idx], cv2.IMREAD_GRAYSCALE)

        if self.augment:
            sample = self.augment(image=image, mask=mask)
            image,mask = sample['image'],sample['mask']
        if self.preprocess:
            sample = self.preprocess(image=image,mask=mask)
            image,mask = sample['image'],sample['mask']

        mask = (mask / 255).astype(np.float32)
        mask = np.expand_dims(mask, axis=0)

        return image,mask

In [ ]:
encoder = "resnet18"
encoder_wts = "imagenet"
activation = "sigmoid"

model = smp.Unet(encoder_name=encoder,activation=activation,encoder_weights=encoder_wts)

preprocess_func = smp.encoders.get_preprocessing_fn(encoder,encoder_wts)

traindata = NervDataset(imagespath = df_train['image'].tolist(),
                            maskspath = df_train['mask'].tolist(),
                            augment = trainaugs(),
                            preprocess = func_for_preprocessing(preprocess_func))

validationdata = NervDataset(imagespath = df_val['image'].tolist(),
                            maskspath = df_val['mask'].tolist(),
                            augment = valaugs(),
                           preprocess = func_for_preprocessing(preprocess_func))

batch_size = 16
trainloader = DataLoader(traindata,batch_size = batch_size,shuffle=True)
valloader = DataLoader(validationdata,batch_size=batch_size,shuffle=False)

In [ ]:
trainmodel = True
epochs = 20
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
loss = smp.utils.losses.DiceLoss()
metrics = [ smp.utils.metrics.IoU(threshold=0.5) ]
optimizer = torch.optim.Adam([dict(params=model.parameters(), lr=0.0001)])
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=1, T_mult=2, eta_min=5e-5)

trainepoch = smp.utils.train.TrainEpoch(model,loss=loss,optimizer=optimizer,metrics=metrics,device=device,verbose=True)
validepoch = smp.utils.train.ValidEpoch(model,loss=loss,metrics=metrics,device=device,verbose=True)

In [ ]:
if trainmodel:
    best_iou_score = 0.0 
    train_logs_list, valid_logs_list = [], []
    for i in range(0,epochs):
        print('\nEpoch: {}'.format(i))
        trainlogs = trainepoch.run(trainloader)
        validlogs = validepoch.run(valloader)
        lr_scheduler.step()
        
        train_logs_list.append(trainlogs)
        valid_logs_list.append(validlogs)
        if best_iou_score < validlogs['iou_score']:
            best_iou_score = validlogs['iou_score']
            if not os.path.exists("./savedmodels"):
                os.mkdir("savedmodels")
            torch.save(model, './savedmodels/best_model_with_segmentation_models_pytorch.pth')
    print("Model Training completed successfully !")

In [ ]:
train_logs_df = pd.DataFrame(train_logs_list)
valid_logs_df = pd.DataFrame(valid_logs_list)
train_logs_df.T

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(train_logs_df.index.tolist(), train_logs_df.iou_score.tolist(),'g-',lw=3, label = 'Train')
plt.plot(valid_logs_df.index.tolist(), valid_logs_df.iou_score.tolist(),'r-' ,lw=3, label = 'Valid')
plt.xlabel('Epochs', fontsize=20)
plt.ylabel('IoU', fontsize=20)
plt.title('IoU', fontsize=20)
plt.legend(loc='best', fontsize=16)
plt.grid()
plt.show()

plt.figure(figsize=(10,4))
plt.plot(train_logs_df.index.tolist(), train_logs_df.dice_loss.tolist(),'g-',lw=3, label = 'Train')
plt.plot(valid_logs_df.index.tolist(), valid_logs_df.dice_loss.tolist(),'r-' ,lw=3, label = 'Valid')
plt.xlabel('Epochs', fontsize=20)
plt.ylabel('Dice', fontsize=20)
plt.title('Dice', fontsize=20)
plt.legend(loc='best', fontsize=16)
plt.grid()
plt.show()

In [ ]:
def rle_encoding(x):
    dots = np.where(x.T.flatten()==1)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b+1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths

best_model = torch.load('./savedmodels/best_model_with_segmentation_models_pytorch.pth')
input_path = '../input/ultrasound-nerve-segmentation/test'
imgs = [f for f in os.listdir(input_path)]
imgs = sorted(imgs, key=lambda s: int(s.split('.')[0]))

encodings = []

for m in tqdm(imgs):
    x = cv2.imread(os.path.join(input_path, m))

    x = valaugs()(image=x)['image']
    x = func_for_preprocessing(preprocess_func)(image=x)['image']

    x_tensor = torch.from_numpy(x).to(device).unsqueeze(0)
    pr_mask = best_model.predict(x_tensor)

    pr_mask = pr_mask.squeeze().cpu().numpy().round().astype(np.uint8)
    pr_mask = albumentations.Resize(height=420,width=580,interpolation=Image.NEAREST)(image=pr_mask)['image']

    encodings.append(rle_encoding(pr_mask))

In [ ]:
with open('submission.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow(["img", "pixels"])
    
    for i, encoding in enumerate(encodings):
        spamwriter.writerow([str(i+1), ' '.join(map(str, encoding))])